<a href="https://colab.research.google.com/github/alexvaroz/data_science_alem_do_basico/blob/master/Regex_para_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Prática de Regex**

Neste notebook exploraremos um exemplo de aplicação de técnicas de expressões regulares em um problema de processamento de linguagem natural.

No caso, foi criada uma peça jurídica fictícia onde constam a identificação de denunciados (nomes e CPFs/CNPJs) e deve-se coletar todos criando organizando-os em uma tabela.

Essa peça pode ser baixada [neste link](https://github.com/alexvaroz/data_science_alem_do_basico/raw/master/toy_denuncia_regex.pdf). Como ela está no formato \*.pdf será utilizada a biblioteca **PdfPlumer** para extrair o conteúdo textual.



In [1]:
# instalação da biblioteca PdfPlumber
!pip install pdfplumber -q

In [2]:
# baixar arquivo da denúncia para o ambiente Colab
!wget https://github.com/alexvaroz/data_science_alem_do_basico/raw/master/toy_denuncia_regex.pdf

--2023-04-10 01:00:40--  https://github.com/alexvaroz/data_science_alem_do_basico/raw/master/toy_denuncia_regex.pdf
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexvaroz/data_science_alem_do_basico/master/toy_denuncia_regex.pdf [following]
--2023-04-10 01:00:40--  https://raw.githubusercontent.com/alexvaroz/data_science_alem_do_basico/master/toy_denuncia_regex.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59069 (58K) [application/octet-stream]
Saving to: ‘toy_denuncia_regex.pdf.2’

toy_denuncia_regex. 100%[===================>]  57.68K  --.-KB/s    in 0.008s  

2023-04-10 01:00:40 

In [3]:
import pdfplumber
import pandas as pd
import re

arquivo = '/content/toy_denuncia_regex.pdf'

In [4]:
pdf = pdfplumber.open(arquivo)

# Quantidade de páginas do documento
len(pdf.pages)

1

In [5]:
txt_denuncia = pdf.pages[0].extract_text()
print(txt_denuncia)

MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REPÚBLICA NO ESTADO
DO RIO DE JANEIRO
Núcleo de Combate à Corrupção
EXCELENTÍSSIMO SENHOR JUIZ FEDERAL DA 7ª VARA FEDERAL CRIMINAL DA
SEÇÃO JUDICIÁRIA DO RIO DE JANEIRO
O Ministério Público Federal (MPF) apresentou denúncia contra 12 empresas,
acusando-as de participar de um esquema de superfaturamento na compra de
medicamentos pelo governo federal. Segundo a denúncia, as empresas - EMS
(CNPJ: 00.000.000/0001-01), Eurofarma (CNPJ: 00.000.000/0002-02), Apsen
(CNPJ: 00.000.000/0003-03), Biolab Sanus (CNPJ: 00.000.000/0004-04), Cristália
(CNPJ: 00.000.000/0005-05), Germed (CNPJ: 00.000.000/0006-06), Hypera
Pharma (CNPJ: 00.000.000/0007-07), Legrand Pharma (CNPJ: 00.000.000/0008-
08), Nova Química (CNPJ: 00.000.000/0009-09), Prati-Donaduzzi (CNPJ:
00.000.000/0010-10), Sanofi (CNPJ: 00.000.000/0011-11) e Teuto Brasileiro (CNPJ:
00.000.000/0012-12) - teriam formado um cartel para manipular as licitações e
aumentar os preços dos medicamentos vendidos

O primeiro passo será a captura do nome das empresas e respectivos CNPJs.

In [6]:
# padrao: <nome da empresa> (CNPJ: <CNPJ>)
padrao = r'[-|e|,]\s([\w*\s?]*[-|\w]*)\s\(CNPJ:\s(\d{2}\.\d{3}\.\d{3}/\d{4}-\n?\d{2})'

Para tratar as informações coletadas, será criado um dataframe:

In [7]:
# Criando um dataframe para abrigar o resultado
df_empresa_cnpj = pd.DataFrame(re.findall(padrao, txt_denuncia), columns = ['EMPRESA', 'CNPJ'])

Importante realizar ajustes nos dados capturados. No caso, serão tratadas as quebras de linha eventualmente presentes na coleta.

In [8]:
# para corrigir os nomes e cnpj
df_empresa_cnpj.EMPRESA = df_empresa_cnpj.EMPRESA.str.replace('\n',' ')
df_empresa_cnpj.CNPJ = df_empresa_cnpj.CNPJ.str.replace('\n','')

Finalizada a coleta dos nomes das empresas e respectivos CNPJs, chega o momento de capturar os citados:

In [9]:
# padrao: <nome da citado> (CPF: <CPF>), <descrição do cargo>
# exemplo: Thiago Martins (CPF: 234.567.891-01), diretor financeiro da Sanofi
padrao_citados = r'[;|m][\s]([\w\s]*)\(CPF:\s(\d{3}\.\d{3}\.\d{3}-\n?\d{2})\),\s([\w\-\s]*)'

In [10]:
re.findall(padrao_citados, txt_denuncia)

[('Carlos Silva ', '012.345.678-90', 'CEO da EMS'),
 ('Júlia Santos\n', '234.567.890-12', 'presidente da Eurofarma'),
 ('Gustavo Lima ', '456.789.012-\n34', 'diretor financeiro da Apsen'),
 ('Luís Vieira ', '678.901.234-56', 'presidente da\nBiolab Sanus'),
 ('Pedro Carvalho ', '890.123.456-78', 'CEO da Cristália'),
 ('Marcelo\nRibeiro ', '123.456.789-00', 'presidente da Germed'),
 ('Paulo Leite ', '345.678.901-23', 'CEO da Hypera Pharma'),
 ('Márcio Costa ', '567.890.123-45', 'diretor financeiro da Legrand Pharma'),
 ('João Paulo Almeida ', '789.012.345-67', 'presidente da Nova Química'),
 ('Joana Santos ', '901.234.567-89', 'CEO da Prati-\nDonaduzzi'),
 ('Thiago Martins ', '234.567.891-01', 'diretor financeiro da Sanofi'),
 ('e\nAntônio Oliveira ', '456.789.012-35', 'presidente do Teuto Brasileiro')]

In [11]:
# Criando um dataframe para abrigar o resultado dos citados
df_citados_cargo = pd.DataFrame(re.findall(padrao_citados, txt_denuncia), 
                                 columns = ['CITADO', 'CPF', 'DESC_CARGO'])

Hora de ajustar os dados coletados:

In [12]:
# Correção das descrições
df_citados_cargo.DESC_CARGO = df_citados_cargo.DESC_CARGO.str.replace('-\n','-')
df_citados_cargo.DESC_CARGO = df_citados_cargo.DESC_CARGO.str.replace('\n',' ')
df_citados_cargo.CITADO = df_citados_cargo.CITADO.str.replace('\n',' ')
df_citados_cargo.CPF = df_citados_cargo.CPF.str.replace('\n','')

Para identificar os nomes de empresa na descrição dos cargos, será utilizada a lista de nomes de empresas coletadas vinculadas ao CNPJ para relacionar com a respectiva descrição:

In [13]:
map_descricao_cargo_empresa = {}
for descricao in df_citados_cargo.DESC_CARGO.values:
  for empresa in df_empresa_cnpj.EMPRESA.values:
    if(re.match(r'([\w\s]*)\sd[ae] {}'.format(empresa),descricao)):
      map_descricao_cargo_empresa[descricao]=(
        re.findall(r'([\w\s]*)\sd[aeo]\s([\w\s\-]*)',descricao)[0])

Criando um dataframe para abrigar o resultado:

In [14]:
df_desc_cargo = pd.DataFrame.from_dict(map_descricao_cargo_empresa, orient='index')
df_desc_cargo.columns = ['CARGO', 'EMPRESA']

In [15]:
df_desc_cargo = df_desc_cargo.reset_index()
df_desc_cargo.rename(columns = {'index':'DESC_CARGO'}, inplace=True)

Com os 3 dataframes criados, chega-se o momento de concluir a rotina integrando os 3.

In [16]:
# Agora podemos integrar os 3 dataframes
final = df_citados_cargo.merge(df_desc_cargo, on = 
                                'DESC_CARGO').merge(df_empresa_cnpj, on='EMPRESA')
final

,CITADO,CPF,DESC_CARGO,CARGO,EMPRESA,CNPJ
0,Carlos Silva,012.345.678-90,CEO da EMS,CEO,EMS,00.000.000/0001-01
1,Júlia Santos,234.567.890-12,presidente da Eurofarma,presidente,Eurofarma,00.000.000/0002-02
2,Gustavo Lima,456.789.012-34,diretor financeiro da Apsen,diretor financeiro,Apsen,00.000.000/0003-03
3,Luís Vieira,678.901.234-56,presidente da Biolab Sanus,presidente,Biolab Sanus,00.000.000/0004-04
4,Pedro Carvalho,890.123.456-78,CEO da Cristália,CEO,Cristália,00.000.000/0005-05
5,Marcelo Ribeiro,123.456.789-00,presidente da Germed,presidente,Germed,00.000.000/0006-06
6,Paulo Leite,345.678.901-23,CEO da Hypera Pharma,CEO,Hypera Pharma,00.000.000/0007-07
7,Márcio Costa,567.890.123-45,diretor financeiro da Legrand Pharma,diretor financeiro,Legrand Pharma,00.000.000/0008-08
8,João Paulo Almeida,789.012.345-67,presidente da Nova Química,presidente,Nova Química,00.000.000/0009-09
9,Joana Santos,901.234.567-89,CEO da Prati-Donaduzzi,CEO,Prati-Donaduzzi,00.000.000/0010-10


O resultado poderia ser exportado para um arquivo no formato .xlsx .

O texto da denúncia foi gerado com o apoio do **ChatGPT**, utilizando o seguinte prompt:

"Como professor universitário, preciso de um exemplo de denuncia do MPF sobre um suposto esquema de superfaturamento na compra de remédios de alto custo envolvendo 12 empresas. Nessa denuncia deve constar os CNPJs e CPFs dos responsáveis por cada empresa."

Que tal criar outros exemplos de texto para praticar??